# HIV Infections in New York City

This analyzes the patterns and disparities of HIV infections in New York City. The aim is to find out how race/ethnicity, age group, gender and neighborhood influence the rate of HIV AIDS contamination.


Importing all required libraries

In [538]:

import requests
import pandas as pd
import json
from pathlib import Path
import sqlite3
import re

Reading the dataset of HIV AIDS Diagnosis by Gender, Neighborhood and Race/Ethnicity. And creating a dataframe called df_Gender

In [539]:

df_Gender = pd.read_csv('Dataset/HIV_AIDS_by_Gender_by_Neighborhood_and_Race_Ethnicity.csv')
df_Gender




,YEAR,Borough,Neighborhood (U.H.F),SEX,RACE/ETHNICITY,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION"
0,2010,NaN,Greenpoint,Male,Black,6,330.4,0,0,5,275.3
1,2011,NaN,Stapleton - St. George,Female,Native American,0,0,0,0,0,0
2,2010,NaN,Southeast Queens,Male,All,23,25.4,5,21.7,14,15.4
3,2012,NaN,Upper Westside,Female,Unknown,0,0,0,0,0,0
4,2013,NaN,Willowbrook,Male,Unknown,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8971,2021,Staten\nIsland,Willowbrook,Male,Asian/Pacific\nIslander,2,24.2,0,0,0,0
8972,2021,Staten\nIsland,Willowbrook,Male,Black,0,0,0,NaN,0,0
8973,2021,Staten\nIsland,Willowbrook,Male,Latino/Hispanic,1,18.2,0,0,0,0
8974,2021,Staten\nIsland,Willowbrook,Male,Other/Unknown,0,0,0,NaN,1,153


Reading the dataset of HIV AIDS Diagnosis by Age, Neighborhood and Race/Ethnicity. And creating a dataframe called df_Age

In [540]:

df_Age = pd.read_csv('Dataset/HIV_AIDS_by_Age_Group_by_Neighborhood_and_Race_Ethnicity.csv')
df_Age

,YEAR,NEIGHBORHOOD,RACE/ETHNICITY,AGE,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION",Borough
0,2013,High Bridge - Morrisania,Native American,30 - 39,0,0,0,0,0,0,NaN
1,2010,East Harlem,Unknown,40 - 49,0,NaN,0,NaN,0,NaN,NaN
2,2011,High Bridge - Morrisania,Unknown,40 - 49,0,NaN,0,NaN,0,NaN,NaN
3,2012,Greenwich Village - SoHo,Native American,50 - 59,0,0,0,0,0,0,NaN
4,2011,Long Island City - Astoria,Multiracial,20 - 29,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21031,2021,Willowbrook,Black,60+,0,0,0,NaN,0,0,Staten\nIsland
21032,2021,Willowbrook,Latino/Hispanic,60+,0,0,0,NaN,0,0,Staten\nIsland
21033,2021,Willowbrook,Other/Unknown,60+,0,0,0,NaN,NaN,NaN,Staten\nIsland
21034,2021,Willowbrook,White,60+,0,0,0,NaN,1,5.6,Staten\nIsland


In the next few lines, I will be cleaning the data. Both datasets will be cleaned separatly before joining them. Lets start by checking the data type we are working with.

In [541]:
# Checking the number of rows & columns in Gender dataset
df_Gender.shape

(8976, 11)

In [542]:
# Checking the number of rows & columns in Age dataset
df_Age.shape

(21036, 11)

In [543]:
#Slicing the number of rows in Age to match Gender
df_Age = df_Age.head(8976).copy()
df_Age.shape


(8976, 11)

In [544]:
# Checking unique values in Age before joining the datasets
pd.unique(df_Age["NEIGHBORHOOD"])


array(['High Bridge - Morrisania', 'East Harlem',
       'Greenwich Village - SoHo', 'Long Island City - Astoria',
       'Upper Eastside', 'West Queens', 'Chelsea - Clinton',
       'Southeast Queens', 'Unknown', 'East New York',
       'East Flatbush - Flatbush', 'Greenpoint', 'Canarsie - Flatlands',
       'Port Richmond', 'Central Harlem - Morningside Heights',
       'Hunts Point - Mott Haven', 'Williamsburg - Bushwick', 'Jamaica',
       'Lower Manhattan', 'Pelham - Throgs Neck', 'Crotona - Tremont',
       'Fordham - Bronx Park', 'Ridgewood - Forest Hills',
       'Downtown - Heights - Park Slope', 'Kingsbridge - Riverdale',
       'Flushing - Clearview', 'Washington Heights - Inwood',
       'Stapleton - St. George', 'Willowbrook',
       'Union Square - Lower Eastside', 'Borough Park',
       'South Beach - Tottenville', 'Fresh Meadows',
       'Bayside - Little Neck', 'Sunset Park', 'Southwest Queens',
       'Upper Westside', 'Coney Island - Sheepshead Bay', 'Rockaway',
    

In [545]:
# Checking unique values in Gender before joining the datasets
pd.unique(df_Gender["Neighborhood (U.H.F)"])

array(['Greenpoint', 'Stapleton - St. George', 'Southeast Queens',
       'Upper Westside', 'Willowbrook', 'East Flatbush - Flatbush',
       'Southwest Queens', 'Fordham - Bronx Park', 'Flushing - Clearview',
       'Chelsea - Clinton', 'Coney Island - Sheepshead Bay', 'Unknown',
       'Gramercy Park - Murray Hill', 'Crotona - Tremont', 'Jamaica',
       'South Beach - Tottenville', 'Union Square - Lower Eastside',
       'Borough Park', 'Port Richmond', 'Greenwich Village - SoHo',
       'East New York', 'Downtown - Heights - Park Slope',
       'Pelham - Throgs Neck', 'Sunset Park',
       'Central Harlem - Morningside Heights', 'West Queens',
       'Bedford Stuyvesant - Crown Heights', 'Fresh Meadows',
       'Bayside - Little Neck', 'High Bridge - Morrisania',
       'Bensonhurst - Bay Ridge', 'Washington Heights - Inwood',
       'Upper Eastside', 'Hunts Point - Mott Haven',
       'Kingsbridge - Riverdale', 'Northeast Bronx',
       'Canarsie - Flatlands', 'East Harlem', 'Lowe

In [546]:
df_Age.dtypes


YEAR                                                                    int64
NEIGHBORHOOD                                                           object
RACE/ETHNICITY                                                         object
AGE                                                                    object
TOTAL NUMBER OF HIV DIAGNOSES                                          object
HIV DIAGNOSES PER 100,000 POPULATION                                   object
TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES                          object
PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES    object
TOTAL NUMBER OF AIDS DIAGNOSES                                         object
AIDS DIAGNOSES PER 100,000 POPULATION                                  object
Borough                                                                object
dtype: object

In [547]:
df_Gender.dtypes

YEAR                                                                    int64
Borough                                                                object
Neighborhood (U.H.F)                                                   object
SEX                                                                    object
RACE/ETHNICITY                                                         object
TOTAL NUMBER OF HIV DIAGNOSES                                          object
HIV DIAGNOSES PER 100,000 POPULATION                                   object
TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES                          object
PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES    object
TOTAL NUMBER OF AIDS DIAGNOSES                                         object
AIDS DIAGNOSES PER 100,000 POPULATION                                  object
dtype: object

Doing some cleaning; handling commas, blank spaces and converting the following  numeric columns from Text to Numbers in both datasets: Total Number of HIV diagnoses, HIV Diagnoses per 100,000 Population, Total Number of Concurrent HIV/AIDS Diagnoses, Proportion of Concurrent HIV/AIDS Diagnoses among all HIV Diagnoses, Total Number of AIDS Diagnoses & AIDS Diagnoses per 100,000 Population



In [548]:
# Converting the grouped by Gender dataset columns to numeric, handling errors and strings with commas

columns_to_convert = [
    'TOTAL NUMBER OF HIV DIAGNOSES',
    'HIV DIAGNOSES PER 100,000 POPULATION',
    'TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES',
    'PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES',
    'TOTAL NUMBER OF AIDS DIAGNOSES',
    'AIDS DIAGNOSES PER 100,000 POPULATION'
]


for col in columns_to_convert:
    df_Gender[col] = pd.to_numeric(df_Gender[col].astype(str).str.replace(",", "").str.strip(), errors='coerce')

print(df_Gender[columns_to_convert].dtypes)

#print(df_Gender[columns_to_convert].head())


TOTAL NUMBER OF HIV DIAGNOSES                                          float64
HIV DIAGNOSES PER 100,000 POPULATION                                   float64
TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES                          float64
PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES    float64
TOTAL NUMBER OF AIDS DIAGNOSES                                         float64
AIDS DIAGNOSES PER 100,000 POPULATION                                  float64
dtype: object


Renaming Neighborhood (U.H.F) column in Gender to match NEIGHBORHOOD in Age

In [549]:
df_Gender.rename(columns={"Neighborhood (U.H.F)": "NEIGHBORHOOD"}, inplace=True)
df_Gender.columns


Index(['YEAR', 'Borough', 'NEIGHBORHOOD', 'SEX', 'RACE/ETHNICITY',
       'TOTAL NUMBER OF HIV DIAGNOSES', 'HIV DIAGNOSES PER 100,000 POPULATION',
       'TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES',
       'PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES',
       'TOTAL NUMBER OF AIDS DIAGNOSES',
       'AIDS DIAGNOSES PER 100,000 POPULATION'],
      dtype='object')

In [550]:
# Converting the grouped by Age dataset columns to numeric, handling errors and strings with commas

columns_to_convert = [
    'TOTAL NUMBER OF HIV DIAGNOSES',
    'HIV DIAGNOSES PER 100,000 POPULATION',
    'TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES',
    'PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES',
    'TOTAL NUMBER OF AIDS DIAGNOSES',
    'AIDS DIAGNOSES PER 100,000 POPULATION'
]


for col in columns_to_convert:
    df_Age[col] = pd.to_numeric(df_Age[col].astype(str).str.replace(",", "").str.strip(), errors='coerce')

print(df_Age[columns_to_convert].dtypes)

#print(df_Age[columns_to_convert].head())



TOTAL NUMBER OF HIV DIAGNOSES                                          float64
HIV DIAGNOSES PER 100,000 POPULATION                                   float64
TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES                          float64
PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES    float64
TOTAL NUMBER OF AIDS DIAGNOSES                                         float64
AIDS DIAGNOSES PER 100,000 POPULATION                                  float64
dtype: object


Age Data preview

In [551]:
pd.set_option('display.max_rows', 50)     # Show up to 100 rows
pd.set_option('display.max_columns', 11)   # Show up to 11 columns
df_Age.head()

#df_Age.iloc[54]


,YEAR,NEIGHBORHOOD,RACE/ETHNICITY,AGE,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION",Borough
0,2013,High Bridge - Morrisania,Native American,30 - 39,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2010,East Harlem,Unknown,40 - 49,0.0,NaN,0.0,NaN,0.0,NaN,NaN
2,2011,High Bridge - Morrisania,Unknown,40 - 49,0.0,NaN,0.0,NaN,0.0,NaN,NaN
3,2012,Greenwich Village - SoHo,Native American,50 - 59,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2011,Long Island City - Astoria,Multiracial,20 - 29,0.0,0.0,0.0,0.0,0.0,0.0,NaN


Gender Data preview

In [552]:
pd.set_option('display.max_rows', 50)     # Show up to 100 rows
pd.set_option('display.max_columns', 11)   # Show up to 11 columns
df_Gender.head()


,YEAR,Borough,NEIGHBORHOOD,SEX,RACE/ETHNICITY,TOTAL NUMBER OF HIV DIAGNOSES,"HIV DIAGNOSES PER 100,000 POPULATION",TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION"
0,2010,NaN,Greenpoint,Male,Black,6.0,330.4,0.0,0.0,5.0,275.3
1,2011,NaN,Stapleton - St. George,Female,Native American,0.0,0.0,0.0,0.0,0.0,0.0
2,2010,NaN,Southeast Queens,Male,All,23.0,25.4,5.0,21.7,14.0,15.4
3,2012,NaN,Upper Westside,Female,Unknown,0.0,0.0,0.0,0.0,0.0,0.0
4,2013,NaN,Willowbrook,Male,Unknown,0.0,0.0,0.0,0.0,0.0,0.0


Neighborhood standardization in both Age & Gender to allow a common joint

In [553]:
# Age Neighborhood standardization
age_neighborhood = [re.sub(r"[\n\r\- ]+", "", string.lower().strip()) for string in pd.unique(df_Age ["NEIGHBORHOOD"])]
age_neighborhood.sort()
print(age_neighborhood)

['all', 'baysidelittleneck', 'bedfordstuyvesantcrownheights', 'bedfordstuyvesantcrownheights', 'bensonhurstbayridge', 'boroughpark', 'canarsieflatlands', 'centralharlemmorningsideheights', 'chelseaclinton', 'coneyislandsheepsheadbay', 'crotonatremont', 'downtownheightsparkslope', 'eastflatbushflatbush', 'eastharlem', 'eastnewyork', 'flushingclearview', 'fordhambronxpark', 'freshmeadows', 'gramercyparkmurrayhill', 'greenpoint', 'greenwichvillagesoho', 'highbridgemorrisania', 'huntspointmotthaven', 'jamaica', 'kingsbridgeriverdale', 'longislandcityastoria', 'lowermanhattan', 'northeastbronx', 'pelhamthrogsneck', 'portrichmond', 'ridgewoodforesthills', 'rockaway', 'southbeachtottenville', 'southeastqueens', 'southwestqueens', 'stapletonst.george', 'sunsetpark', 'unionsquarelowereastside', 'unknown', 'uppereastside', 'upperwestside', 'washingtonheightsinwood', 'westqueens', 'williamsburgbushwick', 'willowbrook']


In [554]:
# Gender Neighborhood standardization
gender_neighborhood = [re.sub(r"[\n\r\- ]+", "", string.lower().strip()) for string in pd.unique(df_Gender ["NEIGHBORHOOD"])]
gender_neighborhood.sort()
print(gender_neighborhood)

['all', 'baysidelittleneck', 'bedfordstuyvesantcrownheights', 'bedfordstuyvesantcrownheights', 'bensonhurstbayridge', 'bensonhurstbayridge', 'boroughpark', 'canarsieflatlands', 'centralharlemmorningsideheights', 'centralharlemmorningsideheights', 'chelseaclinton', 'coneyislandsheepsheadbay', 'coneyislandsheepsheadbay', 'crotonatremont', 'downtownheightsparkslope', 'downtownheightsparkslope', 'eastflatbushflatbush', 'eastflatbushflatbush', 'eastharlem', 'eastnewyork', 'flushingclearview', 'fordhambronxpark', 'fordhambronxpark', 'freshmeadows', 'gramercyparkmurrayhill', 'gramercyparkmurrayhill', 'greenpoint', 'greenwichvillagesoho', 'greenwichvillagesoho', 'highbridgemorrisania', 'highbridgemorrisania', 'huntspointmotthaven', 'huntspointmotthaven', 'jamaica', 'kingsbridgeriverdale', 'kingsbridgeriverdale', 'longislandcityastoria', 'longislandcityastoria', 'lowermanhattan', 'northeastbronx', 'pelhamthrogsneck', 'pelhamthrogsneck', 'portrichmond', 'ridgewoodforesthills', 'ridgewoodforesthi

Neighborhood Column standardised, data sorted in ascendng order and new column created on both Age & Gender

In [555]:
#Function definition allowing the application of the standardised neighborhood values to Age

def clean_age_neighborhood(text):
    if isinstance(text, str):
        return re.sub(r"[\n\r\- ]+", "", text.lower().strip())
    return text


In [556]:
#Creating a new column and sorting values in Age

df_Age["NEIGHBORHOOD_CLEAN"] = df_Age["NEIGHBORHOOD"].astype(str).apply(clean_age_neighborhood)
df_Age = df_Age.sort_values(by="NEIGHBORHOOD_CLEAN", ascending=True)



In [557]:
#Function definition allowing the application of the standardised neighborhood values to Gender
def clean_gender_neighborhood(text):
    if isinstance(text, str):
        return re.sub(r"[\n\r\- ]+", "", text.lower().strip())
    return text

In [558]:
#Creating a new column and sorting values in Gender
df_Gender["NEIGHBORHOOD_CLEAN"] = df_Gender["NEIGHBORHOOD"].astype(str).apply(clean_gender_neighborhood)
df_Gender = df_Gender.sort_values(by="NEIGHBORHOOD_CLEAN", ascending=True)


In [559]:
#Checking Age neighborhood data
print(df_Age["NEIGHBORHOOD_CLEAN"])


8637            all
8624            all
8625            all
8626            all
8627            all
           ...     
2662    willowbrook
6612    willowbrook
7350    willowbrook
5290    willowbrook
4129    willowbrook
Name: NEIGHBORHOOD_CLEAN, Length: 8976, dtype: object


In [560]:
#Checking Gender neighborhood data
print(df_Gender["NEIGHBORHOOD_CLEAN"])

6295            all
3798            all
3797            all
3796            all
3795            all
           ...     
7244    willowbrook
7243    willowbrook
7242    willowbrook
207     willowbrook
8975    willowbrook
Name: NEIGHBORHOOD_CLEAN, Length: 8976, dtype: object


Now that both our datasets have been cleaned, it's time to join them both using SQLite 

In [561]:
import sqlite3

# Saving both Age & Gender Datasets into SQLite
conn = sqlite3.connect("hiv_data.db")
df_Age.to_sql("age_data", conn, if_exists="replace", index=False)
df_Gender.to_sql("gender_data", conn, if_exists="replace", index=False)

# Joining Age & Gender on NEIGHBORHOOD
query = """
SELECT *
FROM age_data AS Age
JOIN gender_data AS Gender
ON Age."NEIGHBORHOOD_CLEAN" = Gender."NEIGHBORHOOD_CLEAN"
"""
Age_Gender_df = pd.read_sql_query(query, conn)

Age_Gender_df.head()


,YEAR,NEIGHBORHOOD,RACE/ETHNICITY,AGE,TOTAL NUMBER OF HIV DIAGNOSES,...,TOTAL NUMBER OF CONCURRENT HIV/AIDS DIAGNOSES,PROPORTION OF CONCURRENT HIV/AIDS DIAGNOSES AMONG ALL HIV DIAGNOSES,TOTAL NUMBER OF AIDS DIAGNOSES,"AIDS DIAGNOSES PER 100,000 POPULATION",NEIGHBORHOOD_CLEAN
0,2016,All,Latino/Hispanic,60+,39.0,...,680.0,20.3,2611.0,31.6,all
1,2016,All,Latino/Hispanic,60+,39.0,...,625.0,19.4,2292.0,27.7,all
2,2016,All,Latino/Hispanic,60+,39.0,...,549.0,18.4,2036.0,24.4,all
3,2016,All,Latino/Hispanic,60+,39.0,...,574.0,20.3,1784.0,21.2,all
4,2016,All,Latino/Hispanic,60+,39.0,...,367.0,16.4,1390.0,19.4,all


Saving Age_Gender table as CSV

In [562]:
Age_Gender_df.to_csv("joined_age_gender_table.csv", index=False)


Sample data to upload to GitHub as it will not accept a large file

In [563]:
# Example: keep only 500 rows
Age_Gender_df.sample(n=500).to_csv("joined_age_gender_table_sample.csv", index=False)


Now moving into data exploration; Where I will look into data descriptive Stats, identify trends and do some group comparisons.

Lets start by finding the total cases by race, age group, sex, and neighborhood

Now looking at the trend in diagnoses. How has this changed over time for each group?

Finally lets do some comparisons to find out which race has the highest infection rate in each neighborhood. Are there gender disparities across neighborhoods? Who are the most affected; young or old age group?